# Hyper-parameter Optimization
Wrap __[hyperopt](https://hyperopt.github.io/hyperopt/)__ into **HyperParameterOpt** to do hyper-parameter optimization. Use random search and logged the parameters have been searched.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from hyperopt import hp, fmin, tpe, Trials, space_eval, STATUS_OK
# from schema.columns_added_filled import LABEL_COL, NUMERICAL_COLS, CATEGORICAL_COLS, LOG_COLS
from schema.columns_added import LABEL_COL, NUMERICAL_COLS, CATEGORICAL_COLS, LOG_COLS
from data_process.data_transform_processor import DataTransformProcessor
from models.nn_models.dnn import DNN
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet
from models.tree_models.lgbm import LGBM
from models.hyperparameter_opt import HyperParameterOpt
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


# Prepare data

In [2]:
df_all = pd.read_csv('/Users/shuyangdu/Desktop/ZillowChallenge/data/df_merged_20171008.csv')

In [3]:
data_processor_dct = {
    'dummy': DataTransformProcessor(
    use_dummy=True, use_scale=True,
    numerical_cols=NUMERICAL_COLS, 
    categorical_cols=CATEGORICAL_COLS,
    log_cols=LOG_COLS, 
    label_col=LABEL_COL,
),
    'tree': DataTransformProcessor(
    numerical_cols=NUMERICAL_COLS, 
    categorical_cols=CATEGORICAL_COLS,
    log_cols=LOG_COLS, 
    label_col=LABEL_COL,
),
    'dummy_pca': DataTransformProcessor(
    use_dummy=True, use_scale=True, use_pca=True,
    numerical_cols=NUMERICAL_COLS, 
    categorical_cols=CATEGORICAL_COLS,
    log_cols=LOG_COLS, 
    label_col=LABEL_COL,
),
    'tree_pca': DataTransformProcessor(
    use_scale=True, use_pca=True,
    numerical_cols=NUMERICAL_COLS, 
    categorical_cols=CATEGORICAL_COLS,
    log_cols=LOG_COLS, 
    label_col=LABEL_COL,
),
}

In [4]:
# use all data_processors to pre process to get categorical idx
for processor in data_processor_dct.values():
    X_all = processor.pre_process(df_all)
y_all = df_all['logerror'].values

# Define search space

## Search space for DNN

In [5]:
space_dnn = {
    'learning_rate': hp.loguniform('learning_rate', -4*np.log(10), -1*np.log(10)),
    'reg': hp.loguniform('reg', -4*np.log(10), 1*np.log(10)),
    'decay': hp.loguniform('decay', -3*np.log(10), -1*np.log(10)),
    'dim_hidden_lst': hp.choice('dim_hidden_lst', [(20,), (40,), (60,), 
                                                   (20,20), (40,40), (60,60),
                                                   (20,20,20), (40,40,40), (60,60,60)]),
    'epochs': 5 + hp.randint('epochs', 6),
}

## Search space for LightGBM

In [6]:
space_gbdt = {
    'learning_rate': hp.loguniform('learning_rate', -2*np.log(10), -1*np.log(10)),  # 1e-2 ~ 1e-1
    'n_estimators': 50 * (8 + hp.randint('n_estimators', 5)),  # 400 ~ 600
    'num_leaves': 20 * (10 + hp.randint('num_leaves', 11)),  # 200 ~ 400
    'lambda_l2': hp.loguniform('lambda_l2', 1*np.log(10), 3*np.log(10)),  # 1e0 ~ 1e3
    'feature_fraction': hp.uniform('feature_fraction', 0.3, 0.8),  #  0.3 ~ 0.8
    'bagging_freq': 10 * (2 + hp.randint('bagging_freq', 7)),  # 20 ~ 80
    'bagging_fraction': hp.uniform('bagging_fraction', 0.7, 1.0),
    'max_bin': 40 * (1 + hp.randint('max_bin', 5)),  # 40 ~ 200
}

space_dart = space_gbdt.copy()
space_dart.update({
    'drop_rate': hp.uniform('drop_rate', 0.0, 0.5),  #  0.0 ~ 0.5
    'skip_drop':hp.uniform('skip_drop', 0.5, 1.0),  #  0.5 ~ 1.0
})
                              
fixed_params_gbdt_l1 = {
    'objective': 'regression_l1',
    'boosting_type': 'gbdt',
    'categorical_feature': data_processor_dct['tree'].categorical_col_idx,
}

fixed_params_gbdt_l2 = {
    'objective': 'regression_l2',
    'boosting_type': 'gbdt',
    'categorical_feature': data_processor_dct['tree'].categorical_col_idx,
}

fixed_params_dart_l1 = {
    'objective': 'regression_l1',
    'boosting_type': 'dart',
    'categorical_feature': data_processor_dct['tree'].categorical_col_idx,
}

## Search space for Linear Regression ElasticNet

In [32]:
space_eln = {
    'alpha': hp.loguniform('alpha', -2*np.log(10), 2*np.log(10)),
    'l1_ratio': hp.uniform('l1_ratio', 0.0, 1.0),
}

# Optimize for hyper-parameters

In [33]:
hyper_opt = HyperParameterOpt(
    model_class=ElasticNet, 
    data_processor=data_processor_dct['dummy'],                           
    search_space=space_eln,
    fixed_params={},
    max_evals=100,
    max_workers=5,
)

In [34]:
%%time
hyper_opt.optimize(X_all, y_all, 12)

Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV finished
Single CV fi

In [35]:
hyper_opt.trial_results

,alpha,l1_ratio,loss,status
0,0.018355,0.138619,0.069259,ok
1,0.017625,0.149046,0.069259,ok
2,0.010066,0.237958,0.069260,ok
3,0.010383,0.252110,0.069260,ok
4,0.025013,0.073535,0.069268,ok
5,0.010600,0.299367,0.069268,ok
6,0.055372,0.061170,0.069271,ok
7,0.019508,0.091124,0.069271,ok
8,0.010119,0.346622,0.069274,ok
9,0.010200,0.355507,0.069278,ok


# Save opt log history

In [36]:
hyper_opt.trial_results.to_csv(
    '/Users/shuyangdu/Desktop/ZillowChallenge/hyper-parameter-opt/eln_added_features_201701010.csv',
)